# Magenta note-seq chords labeler (ver. 0.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (SETUP ENVIRONMENT)

In [ ]:
# @title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/tegridy-tools
!pip install note-seq -U

In [ ]:
# @title Import modules
from note_seq import chord_inference
from note_seq import chords_lib, chord_symbols_lib, sequences_lib
from note_seq.protobuf import music_pb2

# (EXTRACT CHORDS)

In [ ]:
# @title Extract chords
melody_ns1 = midi_io.midi_file_to_note_sequence('/content/tegridy-tools/tegridy-tools/seed2.mid')

# Quantization
melody_ns1 = sequences_lib.quantize_note_sequence(melody_ns, 16)

# Extracting chords
chord_inference.infer_chords_for_sequence(melody_ns1)

# Assuming `sequence` is your NoteSequence object
chords = [annotation.text for annotation in melody_ns1.text_annotations
          if annotation.annotation_type == music_pb2.NoteSequence.TextAnnotation.CHORD_SYMBOL]

#chords_times = [annotation.quantized_step / 64 for annotation in melody_ns1.text_annotations]
chords_times = [annotation.time for annotation in melody_ns1.text_annotations]

print(chords)
print(chords_times)

# (CONVERT CHORDS TO MIDI)

In [ ]:
# @title Convert extracted chords to MIDI
# Assuming `chords` is a list of chord symbols and `chord_times` is a list of chord times
chord_progression = chords_lib.ChordProgression(chords)

# Create a new NoteSequence
sequence = music_pb2.NoteSequence()

# Add the chords to the sequence
for i in range(len(chords)):
    chord_annotation = sequence.text_annotations.add()
    chord_annotation.text = chords[i]
    chord_annotation.time = chords_times[i]
    chord_annotation.annotation_type = music_pb2.NoteSequence.TextAnnotation.CHORD_SYMBOL

    # Get the pitches of the chord symbol
    pitches = chord_symbols_lib.chord_symbol_pitches(chords[i])

    # Sort the pitches in ascending order
    pitches.sort()

    # Add a note for each pitch, setting the octave based on its position in the chord
    for j, pitch in enumerate(pitches):
        note = sequence.notes.add()
        note.pitch = 60 + pitch + (j // len(pitches)) * 12
        note.start_time = chords_times[i]
        note.end_time = chords_times[i + 1] if i + 1 < len(chords_times) else chords_times[i] + 1.0  # or however long you want the note to last
        note.velocity = 80  # or however loud you want the note to be

# Quantize the sequence
sequence = sequences_lib.quantize_note_sequence_absolute(sequence, steps_per_second=64)

# Writing final MIDI
midi_io.sequence_proto_to_midi_file(sequence, 'Generated_Chords_Composition.mid')

# Congrats! You did it! :)